In [8]:
import sys
sys.path.append("..")
from datasets import load_from_disk
from transformers import AutoTokenizer
import numpy as np
from tqdm import tqdm
import torch
import torch.nn.functional as F
from retrieval_model import BertEncoder,RobertaEncoder
from torch import nn
import pandas as pd
from torch.utils.data import (DataLoader,TensorDataset, SequentialSampler)
import pickle
from utils_mrc import seed_everything

In [9]:
seed_everything(42)

In [10]:
# DPR model load
p_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/p_encoder")
q_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/q_encoder")
tokenizer =  AutoTokenizer.from_pretrained("klue/bert-base")
#"kykim/bert-kor-base"

In [11]:
#train_dataset = load_from_disk("/opt/ml/data/train_dataset/train/")
#query_dataset = load_from_disk('/opt/ml/data/test_dataset/validation') # test query
#train_dataset = load_from_disk("/opt/ml/data/train_dataset/new_validation/")
#origin_valid = load_from_disk("/opt/ml/data/train_dataset/validation/")

In [12]:
#wiki_dataset = pd.read_csv('/opt/ml/data/preprocess_wiki_doc.csv')
query_dataset = load_from_disk("/opt/ml/data/train_dataset/validation/")

In [13]:
with open("/opt/ml/data/wiki_id_context_pair.bin", "rb") as f:
    wiki_id_context = pickle.load(f)
# context - doc_id
with open("/opt/ml/data/wiki_context_id_pair.bin", "rb") as f:
    wiki_context_id = pickle.load(f)

In [14]:
wiki_corpus = list(wiki_context_id.keys())

In [15]:
query = query_dataset['question']

In [16]:
eval_batch_size = 32
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)
if torch.cuda.is_available():
    p_encoder.cuda()
    q_encoder.cuda()

# Construt dataloader
train_p_seqs = tokenizer(wiki_corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(
    train_p_seqs["input_ids"],
    train_p_seqs["attention_mask"],
    train_p_seqs["token_type_ids"]
)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(
    valid_dataset,
    sampler=valid_sampler,
    batch_size=eval_batch_size
)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

    epoch_iterator = tqdm(
        valid_dataloader,
        desc="Iteration",
        position=0,
        leave=True
    )
    p_encoder.eval()

    for _, batch in enumerate(epoch_iterator):
        batch = tuple(t.cuda() for t in batch)

        p_inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2]
        }
        
        outputs = p_encoder(**p_inputs).to("cpu").numpy()
        p_embs.extend(outputs)

torch.cuda.empty_cache()

Iteration: 100%|██████████| 1749/1749 [09:41<00:00,  3.01it/s]


In [17]:
train_q_seqs = tokenizer(
    query,
    max_length=64,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

query_dataset = TensorDataset(
    train_q_seqs["input_ids"],
    train_q_seqs["attention_mask"],
    train_q_seqs["token_type_ids"]
)

query_sampler = SequentialSampler(query_dataset)
query_dataloader = DataLoader(
    query_dataset,
    sampler=query_sampler,
    batch_size=eval_batch_size
)

q_embs = []

with torch.no_grad():

    epoch_iterator = tqdm(
        query_dataloader,
        desc="Iteration",
        position=0,
        leave=True
    )
    q_encoder.eval()

    for _, batch in enumerate(epoch_iterator):
        batch = tuple(t.cuda() for t in batch)

        q_inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2]
        }
        
        outputs = q_encoder(**q_inputs).to("cpu").numpy()
        q_embs.extend(outputs)

torch.cuda.empty_cache()
print('done')

Iteration: 100%|██████████| 8/8 [00:00<00:00, 27.04it/s]

done


In [18]:
p_embs = np.array(p_embs)
q_embs = np.array(q_embs)
print(p_embs.shape)
print(q_embs.shape)

(55963, 768)
(240, 768)


In [19]:
with open("/opt/ml/data/dense_embedding.bin", "wb") as file:
    pickle.dump(p_embs,file)

In [30]:
if torch.cuda.is_available():
    p_embs_cuda = torch.Tensor(p_embs).to('cuda')
    q_embs_cuda = torch.Tensor(q_embs).to('cuda')

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))
rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

In [21]:
dense_p_retrieval_result = {}
idx = 0
for i in tqdm(range(len(query))):
    p_list = []
    q = query[i]
    for j in range(100):
        p_list.append(wiki_context_id[wiki_corpus[rank[idx][j]]])
    dense_p_retrieval_result[q] = p_list
    idx += 1

100%|██████████| 240/240 [00:00<00:00, 469.97it/s]


In [34]:
rank.size()

torch.Size([240, 55963])

In [ ]:
#for i in range(rank.size()[0])

In [24]:
with open("/opt/ml/data/dense_valid_retrieval.bin", "wb") as file:
    pickle.dump(dense_p_retrieval_result,file)

# dense_n_retrieval_result = {}
# idx = 0
# for i in tqdm(range(len(query))):
#     p_list = []
#     q = query[i]
#     for j in range(10000,10004):
#         p_list.append(wiki_corpus[rank[idx][j]])
#     dense_n_retrieval_result[q] = p_list
#     idx += 1

In [25]:
idx = 218
rank = 1
print(query[idx])
print()
for rank in range(20):
    print(wiki_id_context[dense_p_retrieval_result[query[idx]][rank]])
    print()

블랙워치의 지도자가 게임에서 불리는 이름은 무엇인가?

블랙 메탈 (Black Metal)이란 헤비 메탈의 하위 장르로써 비명을 지르는 듯한 보컬, 괴상한 기타 소리 등이 특징이다. 유럽에 깊숙히 들어가있는 기독교를 부정하는 것이 특징이었다. 어둠(Black)의 세력인 사탄을 숭배하는 헤비메탈이라고해서 `블랙 메탈'로 불렸다고도 한다. 사운드가 어둡고 음산하며 무거운 것이 특징이나, 심포닉 블랙 메탈이나 바이킹 메탈을 비롯한 여러 하위 장르에서는 다양한 악기를 통해 독특한 분위기를 자아내기도 한다. 블랙메탈은 사실 블랙 사바스에서 이미 나타난 헤비메탈의 일반 대중 음악과 차별화 되는 특성(보컬 위주의 절-후렴에 비해 기타 리프에 내재된 멜로디들의 전개를 통한 곡의 발전)을 가장 강화한 형태의 음악으로, 블랙메탈의 표준과도 같은 버줌이나 엠퍼러 등의 밴드들의 경우 일반 대중 음악과 판이하게 다른 서사적 구조를 사용하고 있다. 그러나 시간이 지나면서 단지 자신들의 문화를 찬양하거나 전쟁을 나타내기 위한 음악들도 나타기 시작했다. 이는 특히 비유럽권 밴드에서 두드러지게 나타나는 특징이며, 유럽의 밴드들 중에서도 나치의 사상으로 기반으로 한 내셔널 소셜리스트 블랙 메탈 장르도 생기기도 하였다. 특히 스칸디나비아 반도에서 전성기를 이루었으며 노르웨이의 버줌, 메이헴, 엠퍼러, 다크스론, 스웨덴의 디섹션, 다크 퓨너럴, 마덕 등이 유명하다. 그 중에서도 실질적으로 블랙메탈의 스타일적 기반을 다진 것은 노르웨이 밴드들이며, 스웨덴에서는 애호가들에게 새크라멘텀이 주요 밴드로 인정받는다. 또한 버줌의 카운트 그리쉬나크와 메이헴의 유로니무스 등이 주도하여 교회 연쇄방화를 일으킨 것과 카운트 그리쉬나크가 유로니우스와의 갈등 끝에 살해한 것이 블랙 메탈에 얽힌 대표적인 일화이다.

유럽의 중세 시대에는 카이트 실드가 사용되었다. 이 방패는 윗 부분이 둥글고 아래 부분은 뾰족하게 모인 모양이었다. 이런 모양의 방패는 말 위에서 사용하기 쉬웠고 내렸을 경우에도 다리를 움직이기 쉬운 장